In [1]:
import random 
import numpy as np
import pandas as pd
import os
import sys
import matplotlib.pyplot as plt
from numpy.linalg import *
from scipy.spatial import distance
#from sklearn import preprocessing
#from sklearn.preprocessing import StandardScaler
np.random.seed(42)  # don't change this line

import base64
import datetime
!pip install recommenders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import sys

import itertools
import pandas as pd

from recommenders.utils.notebook_utils import is_jupyter
from recommenders.datasets import movielens
from recommenders.datasets.python_splitters import python_random_split
from recommenders.datasets.pandas_df_utils import filter_by
from recommenders.evaluation.python_evaluation import (
    rmse, mae, rsquared, exp_var,
    map_at_k, ndcg_at_k, precision_at_k, recall_at_k
)

print("System version: {}".format(sys.version))
print("Pandas version: {}".format(pd.__version__))

System version: 3.9.16 (main, Dec  7 2022, 01:11:51) 
[GCC 9.4.0]
Pandas version: 1.4.4


In [3]:
# from google.colab import files
# uploaded = files.upload()

In [4]:
from google.colab import drive

#use the path that's relevant to you!
drive.mount('/content/drive')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
path = "/content/drive/MyDrive/Senior Year Courses/Spring 2023/CIS 5190/ratings_Electronics.csv"
data = pd.read_csv(path,delimiter=',',names=["userId", "productId", "rating", "timestamp"])
#takes 10s or so

In [6]:
# data = movielens.load_pandas_df(
#     size='100k',
#     header=['UserId', 'MovieId', 'Rating', 'Timestamp']
# )

#get rid of whitespace
data = data.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
data.head()

,userId,productId,rating,timestamp
0,AKM1MP6P0OYPR,0132793040,5.0,1365811200
1,A2CX7LUOHB2NDG,0321732944,5.0,1341100800
2,A2NWSAGRHCP8N5,0439886341,1.0,1367193600
3,A2WNBOD3WNDNKT,0439886341,3.0,1374451200
4,A1GI0U4ZRJA8WN,0439886341,1.0,1334707200


In [7]:
rows_count, columns_count = data.shape
print('Total Number of rows :', rows_count)
print('Total Number of columns :', columns_count)
print(data.dtypes)

Total Number of rows : 7824482
Total Number of columns : 4
userId        object
productId     object
rating       float64
timestamp      int64
dtype: object


In [8]:
unique_userId = data['userId'].nunique()
unique_productId = data['productId'].nunique()
print('Total number of unique Users    : ', unique_userId)
print('Total number of unique Products : ', unique_productId)

Total number of unique Users    :  4201696
Total number of unique Products :  476002


In [9]:
data.isnull().sum()

userId       0
productId    0
rating       0
timestamp    0
dtype: int64

In [10]:
data.describe()

,rating,timestamp
count,7.824482e+06,7.824482e+06
mean,4.012337e+00,1.338178e+09
std,1.380910e+00,6.900426e+07
min,1.000000e+00,9.127296e+08
25%,3.000000e+00,1.315354e+09
50%,5.000000e+00,1.361059e+09
75%,5.000000e+00,1.386115e+09
max,5.000000e+00,1.406074e+09


In [11]:
data['rating'].value_counts()

5.0    4347541
4.0    1485781
1.0     901765
3.0     633073
2.0     456322
Name: rating, dtype: int64

In [21]:
data.shape
#7824482 rows, so grab the first 3912241 rows
#TEMPORAL DATASET SHIFT, ONLY UNCOMMENT WHEN DOING DATA SHIFT
#data=data.sort_values(by='timestamp')[0:3912241]

In [22]:
train, test = python_random_split(data, ratio=0.70, seed=42)

In [23]:
#colnames
#userId       
#productId    
#rating       
#timestamp    

users_ratings = train.groupby(['userId'])['rating'].mean()
users_ratings = users_ratings.to_frame().reset_index()
users_ratings.rename(columns = {'rating': 'AvgRating'}, inplace = True)

users_ratings.head()
#takes about 25s

,userId,AvgRating
0,A00000262KYZUE4J55XGL,5.0
1,A00009661LC9LQPGKJ24G,5.0
2,A0001528BGUBOEVR6T5U,5.0
3,A00018041RRVMCICCAP79,5.0
4,A000187635I595IAVSQLH,5.0


In [24]:
baseline_predictions = pd.merge(test, users_ratings, on=['userId'], how='inner')

baseline_predictions.loc[baseline_predictions['userId'] == 'AKM1MP6P0OYPR'].head()
#takes about 10s

,userId,productId,rating,timestamp,AvgRating


In [25]:
baseline_predictions = baseline_predictions[['userId', 'productId', 'AvgRating']]

cols = {
    'col_user': 'userId',
    'col_item': 'productId',
    'col_rating': 'rating',
    'col_prediction': 'AvgRating',
}

eval_rmse = rmse(test, baseline_predictions, **cols)
eval_mae = mae(test, baseline_predictions, **cols)
eval_rsquared = rsquared(test, baseline_predictions, **cols)
eval_exp_var = exp_var(test, baseline_predictions, **cols)

print("RMSE:\t\t%f" % eval_rmse,
      "MAE:\t\t%f" % eval_mae,
      "rsquared:\t%f" % eval_rsquared,
      "exp var:\t%f" % eval_exp_var, sep='\n')


#takes about 40sec

RMSE:		1.455249
MAE:		0.987732
rsquared:	-0.248466
exp var:	-0.248463


In [26]:
item_counts = train['productId'].value_counts().to_frame().reset_index()
item_counts.columns = ['productId', 'Count']
item_counts.head()

,productId,Count
0,B0019EHU8G,5967
1,B0002L5R78,5342
2,B0074BW614,4388
3,B000LRMS66,4068
4,B003DZ165W,2880


In [29]:
# user_item_col = ['userId', 'productId']

# # Cross join users and items
# test_users = test['userId'].unique()
# user_item_list = list(itertools.product(test_users, item_counts['productId']))
# users_items = pd.DataFrame(user_item_list, columns=user_item_col)

# print("Number of user-item pairs:", len(users_items))

# # Remove seen items (items in the train set) as we will not recommend those again to the users
# users_items_remove_seen = filter_by(users_items, train, user_item_col)

# print("After remove seen items:", len(users_items_remove_seen))

#takes about 
#currently crashes ram..

In [28]:
baseline_recommendations = pd.merge(item_counts, users_items_remove_seen, on=['productId'], how='inner')
baseline_recommendations.head()

NameError: ignored

In [ ]:
k = 5

cols['col_prediction'] = 'Count'

eval_map = map_at_k(test, baseline_recommendations, k=k, **cols)
eval_ndcg = ndcg_at_k(test, baseline_recommendations, k=k, **cols)
eval_precision = precision_at_k(test, baseline_recommendations, k=k, **cols)
eval_recall = recall_at_k(test, baseline_recommendations, k=k, **cols)

print("MAP:\t%f" % eval_map,
      "NDCG@K:\t%f" % eval_ndcg,
      "Precision@K:\t%f" % eval_precision,
      "Recall@K:\t%f" % eval_recall, sep='\n')